In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
180,AFG,Asia,Afghanistan,2020-07-08,33594.0,210.0,936.0,16.0,862.970,5.395,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
302,ALB,Europe,Albania,2020-07-08,3038.0,74.0,81.0,2.0,1055.668,25.714,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
488,DZA,Africa,Algeria,2020-07-08,16879.0,475.0,968.0,9.0,384.917,10.832,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-07,France,168552.0,0.0,29927.0,0.0,0.0,0.0
0,2020-07-07,China,84950.0,0.0,4650.0,0.0,0.0,0.0
0,2020-07-07,Italy,242144.0,0.0,34901.0,0.0,0.0,0.0
0,2020-07-07,Spain,251872.0,0.0,28403.0,0.0,0.0,0.0
0,2020-07-07,United States,2984598.0,0.0,130821.0,0.0,0.0,0.0
0,2020-07-07,World,11828447.0,0.0,542602.0,0.0,0.0,0.0
0,2020-07-07,United Kingdom,286693.0,0.0,44324.0,0.0,0.0,0.0
0,2020-07-07,Germany,197260.0,0.0,9037.0,0.0,0.0,0.0
0,2020-07-07,Iran,246252.0,0.0,11879.0,0.0,0.0,0.0
0,2020-07-07,Turkey,207927.0,0.0,5259.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-08,France,168552.0,168810.0,29927.0,29933.0,-258.0,-6.0
0,2020-07-08,China,84950.0,84917.0,4650.0,4641.0,33.0,9.0
0,2020-07-08,Italy,242144.0,241956.0,34901.0,34899.0,188.0,2.0
0,2020-07-07,Spain,251872.0,252130.0,28403.0,28392.0,-258.0,11.0
0,2020-07-08,United States,2984598.0,2996098.0,130821.0,131480.0,-11500.0,-659.0
0,2020-07-08,World,11828447.0,11801805.0,542602.0,543902.0,26642.0,-1300.0
0,2020-07-08,United Kingdom,286693.0,286349.0,44324.0,44391.0,344.0,-67.0
0,2020-07-08,Germany,197260.0,197341.0,9037.0,9036.0,-81.0,1.0
0,2020-07-08,Iran,246252.0,245688.0,11879.0,11931.0,564.0,-52.0
0,2020-07-08,Turkey,207927.0,207897.0,5259.0,5260.0,30.0,-1.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")